In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout,Activation
from torch.utils.data import TensorDataset, DataLoader
from tensorflow.python.keras import metrics
from tensorflow.python import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from imblearn.over_sampling import SMOTE
import math
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
%matplotlib inline

In [3]:
USE_CUDA=torch.cuda.is_available()
DEVICE=torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
# Epoch과 Batch_size 선언
EPOCHS = 2
BATCH_SIZE = 64

In [5]:
df=pd.read_excel('./optimal_data/'+'Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]
Count=int(322/10)*83


In [6]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        self.input_layer = nn.Linear(6, 128)
        self.hidden_layer1 = nn.Linear(128, 256)
        self.hidden_layer2 = nn.Linear(256, 128)
        self.output_layer   = nn.Linear(128,3)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        out =  self.relu(self.input_layer(x))
        out =  self.relu(self.hidden_layer1(out))
        out =  self.relu(self.hidden_layer2(out))
        out =  self.output_layer(out)
        return out 

In [7]:
model=DNN().to(DEVICE)
# 옵티마이저를 정의합니다. 옵티마이저에는 model.parameters()를 지정해야 합니다.
optimizer    = optim.Adam(model.parameters(), lr=0.01)

# 손실함수(loss function)을 지정합니다. Multi-Class Classification 이기 때문에 CrossEntropy 손실을 지정하였습니다.
loss_fn = nn.CrossEntropyLoss()

### 학습하기

In [8]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        data, target=data.to(DEVICE), target.to(DEVICE)
        #매 반복(iteration) 마다 기울기를 계산하기 위해 zero_grad() 호출
        optimizer.zero_grad()
        # 실제 모델의 예측값(output) 받아오기
        output=model(data)
        #정답 데이터와의 CrossEntropyLoss 계산
        # 손실함수에 output, label 값을 대입하여 손실을 계산합니다.
        loss = loss_fn(output, target)
        #기울기 계산
        loss.backward()
        # 계산된 Gradient를 업데이트 합니다.
        optimizer.step()


### 테스트 하기

In [9]:
def evaluate(model, test_loader):
    # 모델을 평가 모드로 전환
    model.eval()
    # 필요한 변수 초기화
    # Test과정에서의 Loss = test_loss
    # 실제 모델의 예측이 정답과 맞은 횟수 = correct
    test_loss = 0
    correct = 0
    with torch.no_grad(): # 평가 과정에서는 기울기를 계산하지 않으므로, no_grad명시
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)

            # 모든 오차 더하기
            test_loss += loss_fn(output, target).item() * data.size(0)
            
            # 가장 큰 값을 가진 클래스가 모델의 예측입니다.
            # 예측 클래스(pred)과 정답 클래스를 비교하여 일치할 경우 correct에 1을 더합니다.
            pred = output.max(1, keepdim=True)[1]
            # eq() 함수는 값이 일치하면 1을, 아니면 0을 출력.
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    #정확도 계산
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [10]:
for i in range(10):
    #마지막 그룹은 34명
    if i==9:
        X_test=X.iloc[Count*i:-1]
        X_train=X.drop(X.index[Count*i:-1])
        y_test=y.iloc[Count*i:-1]
        y_train=y.drop(y.index[Count*i:-1])
        
    else:
        #모든 그룹은 32명씩 (마지막은 34)
        X_test=X.iloc[Count*i:Count*(i+1)]
        X_train=X.drop(X.index[Count*i:Count*(i+1)])
        y_test=y.iloc[Count*i:Count*(i+1)]
        y_train=y.drop(y.index[Count*i:Count*(i+1)])
    print('SMOTE 적용 전 피처/레이블 데이터 세트: ', X.shape, y.shape)
    print('SMOTE 적용 전 레이블 값 분포: \n', pd.Series(y).value_counts())
    
    # SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train, y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)
    
    print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_test.shape, y_test.shape)
    print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_over).value_counts())

    #모든 데이터 torch로 변환
    X_train = torch.FloatTensor(X_train.to_numpy())
    X_test = torch.FloatTensor(X_test.to_numpy())    
    y_train = torch.LongTensor(y_train.to_numpy())
    y_test = torch.LongTensor(y_test.to_numpy())
    
    
    # train_dataset, test_dataset을 구별하여 정의
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset=TensorDataset(X_test, y_test)

    train_dataloader = DataLoader(train_dataset, batch_size=16,shuffle=False)
    test_dataloader = DataLoader(test_dataset, batch_size=16,shuffle=False)
    for epoch in range(1, EPOCHS + 1):
        train(model, train_dataloader, optimizer)
        test_loss, test_accuracy = evaluate(model, test_dataloader)

        print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

[1] Test Loss: 31.4435, Accuracy: 33.33%
[2] Test Loss: 43.7854, Accuracy: 33.33%
[1] Test Loss: 20.9518, Accuracy: 33.33%
[2] Test Loss: 27.7285, Accuracy: 33.33%
[1] Test Loss: 49.9825, Accuracy: 33.33%


KeyboardInterrupt: 

In [ ]:
# 반복문에서 사용할 수 있도록 iter()함수 적용
dataiter=iter(train_dataloader)

# next()함수로 다음 배치 1개 갖고오기
data, labels=next(dataiter)

In [ ]:
Us

In [ ]:
for label in labels:
    